# 📓 The GenAI Revolution Cookbook

**Title:** CrewAI Agent: Build a Production-Ready Planner-Executor with Memory

**Description:** Ship a production-ready CrewAI agent that plans tasks, validates tools, persists memory, and returns deterministic schema-validated JSON with automatic retries.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Thought: I now can give a great answer

---

## Why This Approach Works

In production, LLM agents fail in predictable ways: they invent facts, forget past context, return inconsistent formats that break parsers, and call tools with wrong inputs. We'll explicitly engineer around those failure modes using planning, persistent memory, strict tool I/O schemas, and final JSON validation with automated retries. If you're curious about why LLMs tend to "forget" as their working memory grows, our article on [Context Rot - Why LLMs "Forget" as Their Memory Grows](/article/context-rot-why-llms-forget-as-their-memory-grows) provides a deeper explanation and practical mitigation strategies.

**Key trade-offs:**

- **Determinism vs flexibility**: We pin temperature to 0.0 for critical paths (planning, tool selection, finalization) to ensure reproducible outputs. This sacrifices creative variation but gains reliability and testability.
- **Schema strictness vs ease of iteration**: Pydantic validation catches malformed outputs early, but requires upfront schema design. The payoff is fewer runtime surprises and cleaner debugging.
- **Memory overhead vs context quality**: Persistent vector memory adds latency and storage, but prevents context loss across sessions and enables long-term learning.
- **Custom orchestration vs framework magic**: We use CrewAI agents for role clarity but handle orchestration manually to maintain full control over validation, retries, and memory writes.

---

## How It Works (High-Level Overview)

**Pipeline data flow:**

1. **Input**: User goal (string)
2. **Memory recall**: Query vector memory for relevant past context
3. **Plan**: Planner agent generates a 3–7 step plan (validated JSON)
4. **Execute loop**: For each step:
   - Recall memory for step-specific context
   - Executor agent selects one tool and arguments (validated JSON)
   - Execute tool with schema validation
   - Extract and persist memory writes
5. **Finalize**: Finalizer agent assembles results, citations, and memory writes into a final answer (validated JSON)
6. **Output**: FinalAnswer object with plan, results, answer text, citations, and memory writes

**Core components:**

- **VectorMemory (ChromaDB + OpenAI embeddings)**: Persistent storage and retrieval of facts
- **Pydantic schemas**: Strict contracts for plans, tool calls, step results, and final answers
- **LLM wrapper with retry logic**: Automatic JSON repair on validation failures
- **Three agents (Planner, Executor, Finalizer)**: Role-based prompting for clarity
- **Safe tools**: Tavily search, sandboxed calculator, HTTP GET JSON with timeout

---

## Setup & Installation

Run this cell first to install dependencies with pinned versions for stability:

In [ ]:
!pip install -q chromadb==0.4.22 openai==1.12.0 pydantic==2.6.1 tavily-python==0.3.3 crewai==0.28.8

Set your API keys securely in Colab. Use the secrets panel (key icon in left sidebar) to store `OPENAI_API_KEY` and `TAVILY_API_KEY`, then run:

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["OPENAI_MODEL"] = "gpt-4o-mini"  # Pin model for determinism

---

## Step-by-Step Implementation

### 1. Define Pydantic Schemas

These schemas enforce strict contracts for plans, tool calls, and outputs. Every LLM response must validate against one of these models.

In [ ]:
from typing import List, Literal, Optional, Dict, Any
from pydantic import BaseModel, Field, HttpUrl

class PlanStep(BaseModel):
    """A single step in a plan."""
    id: int = Field(..., ge=1)
    description: str
    rationale: str

class Plan(BaseModel):
    """A plan consisting of multiple steps."""
    steps: List[PlanStep]

class SearchArgs(BaseModel):
    """Arguments for Tavily search tool."""
    query: str
    max_results: int = Field(5, ge=1, le=10)

class CalcArgs(BaseModel):
    """Arguments for calculator tool."""
    expression: str

class HttpArgs(BaseModel):
    """Arguments for HTTP GET JSON tool."""
    url: HttpUrl
    timeout: int = Field(10, ge=1, le=30)

class ToolCall(BaseModel):
    """Tool selection and arguments."""
    tool: Literal["tavily_search", "calculator", "http_get_json"]
    args: Dict[str, Any]

def validate_tool_args(tc: ToolCall):
    """Validate tool arguments against the correct schema."""
    if tc.tool == "tavily_search":
        SearchArgs(**tc.args)
    elif tc.tool == "calculator":
        CalcArgs(**tc.args)
    elif tc.tool == "http_get_json":
        HttpArgs(**tc.args)
    else:
        raise ValueError(f"Unknown tool: {tc.tool}")

class StepResult(BaseModel):
    """Result of executing a plan step."""
    step_id: int
    tool: str
    input_args: Dict[str, Any]
    output: Any
    notes: Optional[str] = None
    citations: List[str] = Field(default_factory=list)

class FinalAnswer(BaseModel):
    """Final answer schema for the agent."""
    plan: Plan
    results: List[StepResult]
    final_answer: str
    citations: List[str] = Field(default_factory=list)
    memory_writes: List[str] = Field(default_factory=list)

### 2. Implement Safe Tools

Each tool validates inputs and handles errors gracefully. The calculator uses AST parsing to prevent code injection.

In [ ]:
import ast
import operator
import requests
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def tavily_search(args: Dict[str, Any]) -> Dict[str, Any]:
    """Perform a web search using Tavily."""
    params = SearchArgs(**args)
    res = tavily_client.search(params.query, max_results=params.max_results)
    items = []
    for r in res.get("results", []):
        items.append({
            "title": r.get("title"),
            "url": r.get("url"),
            "content": r.get("content")
        })
    return {"results": items[:params.max_results]}

# Allowed AST operations for safe calculator
_ALLOWED_OPS = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.Mod: operator.mod,
    ast.USub: operator.neg,
}

def _eval_expr(node):
    """Safely evaluate a mathematical expression AST node."""
    if isinstance(node, ast.Num):
        return node.n
    if isinstance(node, ast.Constant):
        return node.value
    if isinstance(node, ast.UnaryOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_eval_expr(node.operand))
    if isinstance(node, ast.BinOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_eval_expr(node.left), _eval_expr(node.right))
    raise ValueError("Unsupported expression")

def calculator(args: Dict[str, Any]) -> Dict[str, Any]:
    """Safely evaluate a mathematical expression."""
    params = CalcArgs(**args)
    tree = ast.parse(params.expression, mode="eval")
    value = _eval_expr(tree.body)
    return {"value": value}

def http_get_json(args: Dict[str, Any]) -> Dict[str, Any]:
    """Fetch JSON from a URL with timeout."""
    params = HttpArgs(**args)
    resp = requests.get(str(params.url), timeout=params.timeout)
    resp.raise_for_status()
    content_type = resp.headers.get("Content-Type", "")
    if "application/json" in content_type:
        data = resp.json()
    else:
        data = {"text": resp.text[:2000]}  # Truncate to avoid memory bloat
    return {
        "status": resp.status_code,
        "headers": dict(resp.headers),
        "data": data
    }

### 3. Build LLM Wrapper with Retry Logic

This wrapper prompts the LLM for JSON output and automatically repairs malformed responses up to 3 times.

In [ ]:
import json
import logging
from typing import Type, Any
from openai import OpenAI

logging.basicConfig(level=logging.INFO)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
OPENAI_MODEL = os.environ["OPENAI_MODEL"]

def chat(system: str, user: str, temperature: float = 0.0) -> str:
    """Send a chat completion request to OpenAI."""
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
    )
    return resp.choices[0].message.content

def json_prompt(model: Type[BaseModel], system: str, user: str, max_retries: int = 3) -> Any:
    """
    Prompt LLM for JSON output conforming to a Pydantic schema, with repair loop.
    Retries up to max_retries times if validation fails.
    """
    hint = f"Return ONLY valid JSON that conforms to this schema: {model.model_json_schema()}."
    last_err = None
    content = ""
    for attempt in range(max_retries):
        content = chat(system, f"{user}\n\n{hint}", temperature=0.0)
        try:
            data = json.loads(content)
            return model.model_validate(data)
        except Exception as e:
            last_err = e
            logging.warning(f"JSON validation failed (attempt {attempt+1}): {e}")
            # Attempt repair
            repair = chat(
                "You are a JSON repair tool. Fix to valid JSON exactly.",
                f"Schema:\n{model.model_json_schema()}\n\nInvalid JSON:\n{content}\n\nError:\n{last_err}\n\nReturn only corrected JSON."
            )
            try:
                data = json.loads(repair)
                return model.model_validate(data)
            except Exception as e2:
                last_err = e2
                logging.warning(f"JSON repair failed (attempt {attempt+1}): {e2}")
                continue
    raise ValueError(f"Failed to produce valid JSON after retries. Last error: {last_err}")

### 4. Implement Persistent Vector Memory

We'll persist short-term and long-term memory into a Chroma DB directory. Long-term facts become retrievable context across sessions. We'll use OpenAI's text-embedding-3-small for cost-effective memory indexing and querying. For more on how the placement of critical information in prompts affects LLM recall, see our guide on [Lost in the Middle: Placing Critical Info in Long Prompts](/article/lost-in-the-middle-placing-critical-info-in-long-prompts).

In [ ]:
import uuid
import chromadb
from chromadb.utils import embedding_functions

class VectorMemory:
    """
    Persistent vector memory using ChromaDB and OpenAI embeddings.
    Stores and retrieves facts across sessions.
    """
    def __init__(self, path: str = "./memory_db", collection: str = "crew_memory"):
        self.client = chromadb.PersistentClient(path=path)
        self.embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
            api_key=os.environ["OPENAI_API_KEY"],
            model_name="text-embedding-3-small"
        )
        self.collection = self.client.get_or_create_collection(
            name=collection,
            embedding_function=self.embedding_fn
        )

    def remember(self, text: str, meta: Dict[str, Any] | None = None) -> str:
        """Store a text and optional metadata in vector memory."""
        doc_id = str(uuid.uuid4())
        self.collection.upsert(
            documents=[text],
            metadatas=[meta or {}],
            ids=[doc_id],
        )
        return doc_id

    def recall(self, query: str, k: int = 5) -> List[Dict[str, Any]]:
        """Retrieve top-k relevant memories for a query."""
        res = self.collection.query(query_texts=[query], n_results=k)
        results = []
        for doc, meta, doc_id in zip(
            res.get("documents", [[]])[0],
            res.get("metadatas", [[]])[0],
            res.get("ids", [[]])[0]
        ):
            results.append({"id": doc_id, "text": doc, "meta": meta})
        return results

def memory_to_context(mem: VectorMemory, query: str, k: int = 5) -> str:
    """Retrieve top-k relevant memory entries as context."""
    hits = mem.recall(query, k=k)
    lines = [f"- {h['text']}" for h in hits]
    return "\n".join(lines) if lines else "None"

def extract_memory_writes(result: StepResult) -> List[str]:
    """Extract memory write candidates from a step result."""
    writes = []
    if result.tool == "tavily_search":
        for r in result.output.get("results", []):
            title, url = r.get("title"), r.get("url")
            if title and url:
                writes.append(f"{title} -> {url}")
    if result.tool == "http_get_json":
        url = result.input_args.get("url")
        status = result.output.get("status")
        if url and status == 200:
            writes.append(f"Fetched JSON OK from {url}")
    return writes

### 5. Define CrewAI Agents

These agents provide role-based prompting for planning, execution, and finalization.

In [ ]:
from crewai import Agent

planner_agent = Agent(
    role="Planner",
    goal="Decompose complex requests into minimal, sequential steps to solve the user goal.",
    backstory="You are a meticulous project planner who writes terse, unambiguous plans.",
    verbose=True,
    allow_delegation=False,
)

executor_agent = Agent(
    role="Executor",
    goal="For each step, choose one tool and specify exact arguments to complete the step.",
    backstory="You are precise and only call tools you truly need, with correct arguments.",
    verbose=True,
    allow_delegation=False,
)

finalizer_agent = Agent(
    role="Finalizer",
    goal="Assemble the results and write a concise, accurate final answer with citations.",
    backstory="You summarize clearly, include sources, and keep to the requested schema.",
    verbose=True,
    allow_delegation=False,
)

### 6. Implement Planner

The Planner agent generates a structured plan with 3–7 steps, each validated against the Plan schema.

In [ ]:
PLAN_SYSTEM = (
    "You break down goals into numbered steps with minimal scope per step. "
    "Prefer quick wins. Avoid redundant steps. Use 3-7 steps."
)

def make_plan(user_goal: str, memory_context: str) -> Plan:
    """Prompt the Planner agent to create a Plan."""
    user = (
        f"User goal:\n{user_goal}\n\n"
        f"Relevant memory:\n{memory_context}\n\n"
        "Output a Plan with steps: each has id, description, rationale."
    )
    return json_prompt(Plan, PLAN_SYSTEM, user)

### 7. Implement Executor with Auto-Correction

The Executor selects and executes tools. If a tool fails, it retries with corrected arguments up to 3 times.

In [ ]:
TOOL_DOC = """
Available tools:
- tavily_search(query: str, max_results: int=5)
- calculator(expression: str)
- http_get_json(url: http(s) URL, timeout: int=1..30)
"""

EXEC_SYSTEM = (
    "You are a strict tool selector. For the provided step, return exactly one tool to call and its args."
)

def select_tool(step_text: str, context: str) -> ToolCall:
    """Prompt the Executor agent to select a tool and arguments."""
    user = (
        f"Step:\n{step_text}\n\nContext:\n{context}\n\n"
        f"{TOOL_DOC}\n\nReturn a ToolCall."
    )
    return json_prompt(ToolCall, EXEC_SYSTEM, user)

def run_tool(tc: ToolCall) -> StepResult:
    """Validate and execute the selected tool."""
    validate_tool_args(tc)
    if tc.tool == "tavily_search":
        out = tavily_search(tc.args)
        cits = [r["url"] for r in out.get("results", []) if r.get("url")]
    elif tc.tool == "calculator":
        out = calculator(tc.args)
        cits = []
    elif tc.tool == "http_get_json":
        out = http_get_json(tc.args)
        cits = []
    else:
        raise ValueError(f"Unknown tool: {tc.tool}")
    return StepResult(
        step_id=0,
        tool=tc.tool,
        input_args=tc.args,
        output=out,
        citations=cits
    )

def robust_execute(step_desc: str, context: str, max_attempts: int = 3) -> StepResult:
    """
    Execute a step with automatic correction on tool failure.
    Retries up to max_attempts times if tool execution fails.
    """
    last_error = None
    for attempt in range(max_attempts):
        error_hint = f"\nPrevious error: {last_error}" if last_error else ""
        tc = select_tool(f"{step_desc}{error_hint}", context)
        try:
            return run_tool(tc)
        except Exception as e:
            last_error = str(e)
            logging.warning(f"Tool execution failed (attempt {attempt+1}): {last_error}")
            continue
    raise RuntimeError(f"Execution failed after retries. Last error: {last_error}")

### 8. Implement Finalizer

The Finalizer assembles results, citations, and memory writes into a validated FinalAnswer.

In [ ]:
FINALIZE_SYSTEM = (
    "You produce concise, accurate answers. Include citations whenever you relied on sources."
)

def finalize(plan: Plan, results: List[StepResult], user_goal: str, memory_writes: List[str]) -> FinalAnswer:
    """Prompt the Finalizer agent to assemble the final answer."""
    user = (
        f"User goal:\n{user_goal}\n\n"
        f"Plan:\n{plan.model_dump_json(indent=2)}\n\n"
        f"Results:\n[{', '.join(r.model_dump_json() for r in results)}]\n\n"
        "Compose final_answer (concise but complete), gather unique citations, "
        "and include memory_writes (as provided) in the output."
    )
    return json_prompt(FinalAnswer, FINALIZE_SYSTEM, user)

### 9. Orchestrate the Full Pipeline

This function runs the planner–executor–finalizer pipeline with memory and logging.

In [ ]:
def run_agent(user_goal: str) -> FinalAnswer:
    """
    Run the full agent pipeline: plan, execute, finalize.
    Returns a validated FinalAnswer with plan, results, answer, citations, and memory writes.
    """
    mem = VectorMemory()
    
    # 1) Recall memory for planning
    plan_context = memory_to_context(mem, user_goal, k=5)
    plan = make_plan(user_goal, plan_context)
    logging.info(f"Plan created with {len(plan.steps)} steps")

    # 2) Execute steps with auto-correction
    step_results: List[StepResult] = []
    pending_writes: List[str] = []
    for step in plan.steps:
        ctx = memory_to_context(mem, f"{user_goal} | {step.description}", k=5)
        sr = robust_execute(step.description, ctx)
        sr.step_id = step.id
        step_results.append(sr)
        
        # Memory writes
        writes = extract_memory_writes(sr)
        for w in writes:
            mem.remember(w, meta={"step_id": step.id, "goal": user_goal})
        pending_writes.extend(writes)
        logging.info(f"Step {step.id}: tool={sr.tool}, writes={writes}")

    # 3) Finalize
    answer = finalize(plan, step_results, user_goal, pending_writes)
    logging.info("Final answer assembled")
    return answer

---

## Run and Validate

Run the agent with a realistic goal and print the validated JSON output:

In [ ]:
goal = "Compare the current price of Bitcoin to last week's average and compute the percentage change."
result = run_agent(goal)
print(result.model_dump_json(indent=2))

**Expected output structure:**

```json
{
  "plan": {
    "steps": [
      {"id": 1, "description": "Search for current Bitcoin price", "rationale": "..."},
      {"id": 2, "description": "Search for last week's average Bitcoin price", "rationale": "..."},
      {"id": 3, "description": "Calculate percentage change", "rationale": "..."}
    ]
  },
  "results": [
    {"step_id": 1, "tool": "tavily_search", "input_args": {...}, "output": {...}, "citations": [...]},
    {"step_id": 2, "tool": "tavily_search", "input_args": {...}, "output": {...}, "citations": [...]},
    {"step_id": 3, "tool": "calculator", "input_args": {...}, "output": {...}, "citations": []}
  ],
  "final_answer": "Bitcoin is currently $X. Last week's average was $Y. The percentage change is Z%.",
  "citations": ["https://...", "https://..."],
  "memory_writes": ["Bitcoin price $X -> https://...", "Last week average $Y -> https://..."]
}
```

**Test memory persistence:**

Run a second query that relies on the first query's memory:

In [ ]:
goal2 = "What was the Bitcoin price I looked up earlier?"
result2 = run_agent(goal2)
print(result2.model_dump_json(indent=2))

The agent should recall the Bitcoin price from memory without re-searching.

**Test auto-correction:**

Force a tool failure by sabotaging the input, then observe retry logs:

In [ ]:
# This will fail and retry with corrected arguments
goal3 = "Calculate the result of 10 divided by zero"
try:
    result3 = run_agent(goal3)
except Exception as e:
    print(f"Expected failure: {e}")

---

## Conclusion

You've built a production-oriented planner–executor agent with persistent memory, schema-validated JSON, and automatic error correction. The system decomposes goals into steps, selects and executes tools with strict validation, persists learnings across sessions, and assembles final answers with citations.

**Key decisions:**

- **Pydantic schemas** enforce strict contracts and catch errors early
- **Persistent vector memory** prevents context loss and enables long-term learning
- **Automatic retry loops** handle LLM hallucinations and tool failures gracefully
- **Temperature 0.0** ensures deterministic, reproducible outputs for testing

For deterministic behavior, pin your OpenAI model (e.g., gpt-4o-mini) and run with temperature 0.0 in json_prompt and critical prompts. This reduces drift in tool selections and JSON structure and makes test snapshots reliable across runs and environments. If you're weighing the tradeoffs between deploying small versus large language models in production, our article [Small vs Large Language Models](/article/small-language-models-vs-large-language-models-when-to-use-each) explores when each is most effective, including cost and latency considerations.

**Next steps:**

- **Harden the service endpoint**: Wrap `run_agent` in a FastAPI route with rate limits, authentication, and request validation
- **Add structured logging**: Use Python's logging module with JSON formatters and correlation IDs to trace plan → tool calls → retries → memory writes
- **Keep the tool catalog minimal**: Start with 3–5 tools and expand only when you have clear use cases. Each tool adds validation overhead and increases the chance of incorrect selections
- **Integrate CrewAI hierarchical process**: Use CrewAI's Task and Crew to manage agent coordination if you need more complex workflows. See the optional snippet below for a starting point
- **Monitor and version schemas**: Track schema changes in version control and test backward compatibility when updating Pydantic models
- **Add OAuth tool integrations**: Extend tools to support authenticated APIs (e.g., Google Calendar, Slack) with secure token management

**Optional: CrewAI hierarchical process**

If you want to use CrewAI's built-in task management, define tasks and a crew:

In [ ]:
from crewai import Crew, Process, Task

plan_task = Task(
    description="Create a 3-7 step plan for the user goal with minimal steps.",
    expected_output="A JSON plan with steps: id, description, rationale.",
    agent=planner_agent,
)

crew = Crew(
    agents=[planner_agent, executor_agent, finalizer_agent],
    tasks=[plan_task],
    process=Process.hierarchical,
    verbose=True,
)

# crew.kickoff()  # Use the validation+memory wrappers shown above for production output

This approach delegates orchestration to CrewAI but requires adapting the validation and memory logic to fit CrewAI's task output format.